In [77]:
import pandas as pd

In [78]:
path1 = "COVID19VaccDosesAdministered.csv"
path2 = "COVID19FullyVaccPersons.csv"

In [79]:
#deliv = verimpft, admin = komplett geimpfte personen
df_deliv = pd.read_csv(path1)
df_admin = pd.read_csv(path2)

## Nur jüngstes Datum

In [80]:
df_deliv_Mai = df_deliv[df_deliv["date"] == "2021-03-01"]
df_admin_Mai = df_admin[df_admin["date"] == "2021-03-01"]

In [81]:
#grosse Kantone rausfiltern
df_deliv_MaiGross = df_deliv_Mai[(df_deliv_Mai["pop"] > 800000) & (df_deliv_Mai["pop"] < 1600000)]
df_admin_MaiGross = df_admin_Mai[(df_admin_Mai["pop"] > 800000) & (df_admin_Mai["pop"] < 1600000)]

In [82]:
df_admin_Mai.columns

Index(['date', 'geoRegion', 'entries', 'pop', 'sumTotal', 'per100Persons',
       'per100Persons_mean7d', 'per100PersonsTotal', 'mean7d', 'type',
       'version', 'granularity'],
      dtype='object')

In [83]:
# weglöschen der unnötigen Zahlen
df_deliv_MaiGross = df_deliv_MaiGross.drop(['type', 'version', 'granularity'], axis = 1)
df_admin_MaiGross = df_admin_MaiGross.drop(['type',
       'version', 'granularity'], axis = 1)

In [84]:
#index reseten, um die beiden DFs verbinden zu können
df_deliv_MaiGross.reset_index(drop = True, inplace = True)
df_deliv_MaiGross.rename(columns={"sumTotal": "SumVerimpft"}, inplace = True)
df_deliv_MaiGross

,date,geoRegion,entries,pop,SumVerimpft,per100Persons,per100Persons_mean7d,per100PersonsTotal,mean7d
0,2021-03-01,BE,3139.0,1039474,91303,0.30,0.22,8.78,2271.43
1,2021-03-01,VD,2519.0,805098,82732,0.31,0.21,10.28,1726.29
2,2021-03-01,ZH,1812.0,1539275,122022,0.12,0.19,7.93,2912.43


In [85]:
df_admin_MaiGross.reset_index(drop = True, inplace = True)
df_admin_MaiGross.rename(columns={"sumTotal": "SumKomplett"}, inplace = True)
df_admin_MaiGross

,date,geoRegion,entries,pop,SumKomplett,per100Persons,per100Persons_mean7d,per100PersonsTotal,mean7d
0,2021-03-01,BE,2348.0,1039474,27066,0.23,0.16,2.60,1615.00
1,2021-03-01,VD,1099.0,805098,26283,0.14,0.10,3.26,843.00
2,2021-03-01,ZH,1566.0,1539275,38195,0.10,0.15,2.48,2369.43


In [86]:
df_erstimpfungen = pd.concat([df_deliv_MaiGross, df_admin_MaiGross["SumKomplett"]], axis=1, join="outer")
df_erstimpfungen["Erstimpfung"] = df_erstimpfungen["SumVerimpft"] - (2*df_erstimpfungen["SumKomplett"])
df_erstimpfungen["2te_inprozent"] = (100/df_erstimpfungen["pop"] * df_erstimpfungen["SumKomplett"]).round(1)
df_erstimpfungen["1te_inprozent"] = (100/df_erstimpfungen["pop"] * df_erstimpfungen["Erstimpfung"]).round(1)

In [87]:
df_erstimpfungen = df_erstimpfungen.drop(['entries', 'pop', 'SumVerimpft', 'per100Persons',
       'per100Persons_mean7d', 'per100PersonsTotal', 'mean7d', 'SumKomplett',
       'Erstimpfung'], axis = 1)

In [88]:
df_erstimpfungen

,date,geoRegion,2te_inprozent,1te_inprozent
0,2021-03-01,BE,2.6,3.6
1,2021-03-01,VD,3.3,3.7
2,2021-03-01,ZH,2.5,3.0


In [89]:
#Exportieren Excel
df_erstimpfungen.to_excel("3_1und2_März.xlsx") 